In [ ]:
# Imports
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats

plt.rcParams['figure.figsize'] = (15.0, 8.0) # set default size of plots
plt.rcParams['figure.facecolor'] = 'white'

pd.set_option('display.max_rows', None)

matplotlib.rcParams.update({'font.size': 15})

In [ ]:
TIME_RETRIEVAL_REE = 0
TIME_RETRIEVAL_TEE = 10
TIME_RETRIEVAL_REE_WASM = 0
TIME_RETRIEVAL_TEE_WASM = 13

In [ ]:
def compute_mean_std(name, time):
    df = pd.DataFrame(columns = ["Name","Time"])
    df["Name"] = name
    df["Time"] = time
    
    result = pd.DataFrame(columns = ["Mean","Std"])
    result["Mean"] = df.groupby("Name")["Time"].median() #df.groupby("Size")["Time"].apply(stats.gmean)
    result["Std"] = df.groupby("Name")["Time"].std() #df.groupby("Size")["Time"].apply(stats.gstd)
    
    return result

def formatForLatex(n, digits_round = 3):
    roundedAsString = str(round(n, digits_round))
    if "." in roundedAsString:
        roundedAsString = roundedAsString.rstrip("0")
        roundedAsString = roundedAsString.rstrip(".")
    return f"{roundedAsString}"

In [ ]:
#
## TEE (interpreter) dataset
#

In [ ]:
# Import the dataset
tee_inter_raw = pd.read_csv("../logs/polybench/tee-inter.csv", names = ["Name","Time"])
tee_inter_raw["Name"] = tee_inter_raw["Name"].str.replace(".wasm", "")

tee_inter_raw

In [ ]:
# Adapt the time according the retrieval of the time
tee_inter = tee_inter_raw

tee_inter["Time"] = tee_inter["Time"] - 2*TIME_RETRIEVAL_TEE_WASM/2

tee_inter

In [ ]:
#
## TEE (AoT) dataset
#

In [ ]:
# Import the dataset
tee_aot_raw = pd.read_csv("../logs/polybench/tee-aot.csv", names = ["Name","Time"])
tee_aot_raw["Name"] = tee_aot_raw["Name"].str.replace(".aot", "")

tee_aot_raw.drop(tee_aot_raw.index[len(tee_inter.index):], inplace = True)

tee_aot_raw

In [ ]:
# Adapt the time according the retrieval of the time
tee_aot = tee_aot_raw

tee_aot["Time"] = tee_aot["Time"] - 2*TIME_RETRIEVAL_TEE_WASM/2

tee_aot

In [ ]:
#
## Export
#

In [ ]:
# Normalize the results based on REE (AoT) time
tee_inter_normalized = pd.DataFrame()
tee_aot_normalized = pd.DataFrame()

for col in tee_aot:
    if col == "Name":
        tee_inter_normalized[col] = tee_inter[col]
        tee_aot_normalized[col] = tee_aot[col]
    else:
        tee_inter_normalized[col] = tee_inter[col] / tee_aot[col]
        tee_aot_normalized[col] = 1
    
tee_inter_normalized

In [ ]:
# Compute the mean and std
tee_inter_vs_aot_export = pd.DataFrame()

tee_inter_vs_aot_export = tee_inter_vs_aot_export.append(compute_mean_std(tee_inter_normalized["Name"], tee_inter_normalized["Time"]))
tee_inter_vs_aot_export

In [ ]:
#
## Temp visualisation
#

In [ ]:
df = pd.DataFrame({'interpreter': tee_inter_export["Mean"],
                   'AoT': tee_aot_export["Mean"]}, index=tee_aot_export.index.values)
ax = df.plot.bar()
ax.set_ylim(0.9,200)

In [ ]:
tee_inter_mean_of_mean = tee_inter_export["Mean"].median()
tee_inter_mean_of_mean

In [ ]:
# Export CSV
#
# File format: experiment_name, mean, stddev

tee_inter_vs_aot_export.to_csv('../logs/polybench/inter-vs-aot.csv')

In [39]:
# Export LaTeX

prefix = "InterVsAot"

def write(file, column_name, number, digits_round = 3):
    f.write(f"\\def\\bm{prefix}{column_name}{{{formatForLatex(number, digits_round)}}}\n")

f = open('../logs/polybench/inter-vs-aot-export.tex', "w")
write(f, "GlobalSpeedUp", tee_inter_mean_of_mean, 0)
f.close()